In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d harshghadiya/kidneystone
!unzip \*.zip && rm *.zip

  0% 0.00/1.89k [00:00<?, ?B/s]
100% 1.89k/1.89k [00:00<00:00, 3.36MB/s]
Archive:  kidneystone.zip
  inflating: kidney-stone-dataset.csv  


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data
data = pd.read_csv('/content/kidney-stone-dataset.csv')  # Replace '/path/to/your/dataset.csv' with the actual path

# Step 2: Prepare Data
X = data[['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']]
y = data['target']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Train K-means clustering model
n_clusters = 5  # Number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_train_scaled)

# Step 6: Apply KNN within each cluster
knn_models = []
for cluster_id in range(n_clusters):
    cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
    if len(cluster_indices) > 0:
        knn = KNeighborsClassifier(n_neighbors=5)  # Example: K=5
        knn.fit(X_train_scaled[cluster_indices], y_train.iloc[cluster_indices])
        knn_models.append(knn)

# Step 7: Predict using the trained KMKNN models
y_pred = []
for x_test_point in X_test_scaled:
    # Find the nearest cluster center
    nearest_cluster_index = kmeans.predict([x_test_point])[0]
    # Get indices of data points in the nearest cluster
    nearest_cluster_indices = np.where(kmeans.labels_ == nearest_cluster_index)[0]
    # Extract data points and labels within the nearest cluster
    X_nearest_cluster = X_train_scaled[nearest_cluster_indices]
    y_nearest_cluster = y_train.iloc[nearest_cluster_indices]
    # Apply KNN to data points within the nearest cluster
    knn_model = knn_models[nearest_cluster_index]
    predicted_class = knn_model.predict([x_test_point])[0]
    y_pred.append(predicted_class)

# Step 8: Evaluate the KMKNN model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Display evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Display classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.6666666666666666
Precision: 0.7000000000000001
Recall: 0.6666666666666666
F1 Score: 0.6753246753246753

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.67      0.73        12
           1       0.50      0.67      0.57         6

    accuracy                           0.67        18
   macro avg       0.65      0.67      0.65        18
weighted avg       0.70      0.67      0.68        18

